In [16]:
using Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/Desktop/Work/Research/LEGO-project/LEGO-3D_Dyn`


In [17]:
import ForwardDiff as FD
using LinearAlgebra
using Plots

### Newton's method

In [10]:
function newton_step(func::Function, x0::Vector)::Vector
  # function value at current point
  f = func(x0)
  # check if function is scalar or vector and calculate the Jacobian accordingly
  if length(f) == 1
    J = FD.gradient(func, x0)
    return x0 .- f[1] / J[1]
  else
    J = FD.jacobian(func, x0)
    return x0 - J \ f
  end
end

function newton_root_finding(func::Function, x0::Vector; tol=1e-6, max_iter=100)::Vector
  x = x0
  for i in 1:max_iter
    # take newton step
    x_new = newton_step(func, x)
    if func(x_new) < tol

      return x_new
    end
    x = x_new
  end
  throw("Max iterations reached")
  return x
end

newton_root_finding (generic function with 1 method)

### Quaternions

In [14]:
function hat(v::Vector)::Matrix
  # generate skew symmetric matrix from vector
  return [0 -v[3] v[2];
    v[3] 0 -v[1];
    -v[2] v[1] 0]
end

function L(Q::Vector)::Matrix
  # left multiply matrix of quaternion Q
  s = Q[1] # scalar part
  v = Q[2:end] # vector part
  return [s -v'; v hat(v)+s*I]
end

function R(Q::Vector)::Matrix
  # right multiply matrix of quaternion Q
  s = Q[1] # scalar part
  v = Q[2:end] # vector part
  return [s -v'; v hat(v)-s*I]
end

H = [zeros(1, 3); I]; # hat map for vectors

T = Diagonal([1; -ones(3)]); # quaternion conjugation matrix

function G(Q::Vector)::Matrix
  # generate attitude jacobian from quaternion Q
  return L(Q) * H
end

function quat2rot(Q::Vector)::Matrix
  # convert quaternion to rotation matrix
  return H' * L(Q) * R(Q) * H
end

quat2rot (generic function with 1 method)

### Dynamics

In [ ]:
rigidbody_params = (m=1.0, J=Diagonal([1.0, 1.0, 1.0]), g=9.81) # rigidbody parameters

function states2tuple(states::Vector)::NamedTuple
  """
  # Separate state vectors into position, and attitude

  # Arguments
  - `states`: Vector containing states of the system

  # Returns
  - `states_tuple`: NamedTuple containing separated states
  """
  qᴺ = states[1:3] # position of the rigidbody in world frame

  if length(states) == 3
    states_tuple = (qᴺ = qᴺ)
  elseif length(states) == 7
    ᴺQᴮ = states[4:7] # orientation of the rigidbody in inertial frame
    states_tuple = (qᴺ=qᴺ, ᴺQᴮ=ᴺQᴮ) # separated states
  else
    throw("Invalid state vector")
  end

  return states_tuple
end

function potential_energy(state::Vector, params_rb::NamedTuple)::Float64s
  """
  # Calculate potential energy of the system

  # Arguments
  - `state`: Vector containing states of the system
  - `params_rb`: NamedTuple containing rigidbody parameters

  # Returns
  - `U`: potential energy of the system
  """
  q = states2tuple(state).qᴺ
  z = q[3]
  m, g = params_rb.m, params_rb.g
  U = m * g * z
  return U
end

# left momentum term (at tₖ₊₁) of the linear discrete euler lagrange equation
function D2Ld(state1::Vector, state2::Vector, h::Float64, params_rb::NamedTuple)::Vector
  """
  # Calculate left momentum term of the discrete euler lagrange equation

  # Arguments
  - `state1`: Vector containing states of the system at time t
  - `state2`: Vector containing states of the system at time t + h
  - `h`: time step
  - `params_rb`: NamedTuple containing rigidbody parameters

  # Returns
  - `p⁺`: left momentum term
  """
  q1, q2 = states2tuple(state1).qᴺ, states2tuple(state2).qᴺ
  m = params_rb.m
  q̄ = (q1 + q2) / 2 # midpoint position
  v̄ = (q2 - q1) / h # average velocity
  ∇U = FD.gradient(state -> potential_energy(state, params_rb), q_mid)  
  p⁺ = m * v_avg - h/2 * ∇U # left momentum term
  return p⁺
end

# right momentum term (at tₖ) of the linear discrete euler lagrange equation
function D1Ld(state1::Vector, state2::Vector, h::Float64, params_rb::NamedTuple)::Vector
  """
  # Calculate right momentum term of the discrete euler lagrange equation

  # Arguments
  - `state1`: Vector containing states of the system at time t
  - `state2`: Vector containing states of the system at time t + h
  - `h`: time step
  - `params_rb`: NamedTuple containing rigidbody parameters

  # Returns
  - `_p⁻`: right momentum term
  """
  q1, q2 = states2tuple(state1).qᴺ, states2tuple(state2).qᴺ
  m = params_rb.m
  q̄ = (q1 + q2) / 2 # midpoint position
  v̄ = (q2 - q1) / h # average velocity
  ∇U = FD.gradient(state -> potential_energy(state, params_rb), q_mid)  
  _p⁻ = -m * v_avg - h/2 * ∇U # negative of right momentum term
  return _p⁻
end

function linear_momentum_DLE(momentum1::Vector, state2::Vector, state3::Vector, cg_force1::Vector, cg_force2::Vector, λ::Vector, params_rb::NamedTuple, h::Float64)::Vector
  """
  # Calculate discrete Lagrangian of the system

  # Arguments
  - `state1`: Vector containing states of the system at time tₖ
  - `state2`: Vector containing states of the system at time tₖ + h
  - `forcing1`: Vector containing forcing terms at time tₖ - h/2
  - `forcing2`: Vector containing forcing terms at time tₖ + h/2
  - `λ`: lagrange multiplier
  - `params_rb`: NamedTuple containing rigidbody parameters
  - `h`: time step

  # Returns
  - `lm_DEL`: residual of the discrete linear momentum Lagrangian of the system using midpoint integration
  """
  lm_DEL = momentum1 + D1Ld(state2, state3, h, params_rb) + h/2 * (cg_force1 + cg_force2)
  return lm_DEL
end

# function rotational_DEL(state_1::NamedTuple, state_2::NamedTuple, forcing_1::NamedTuple, forcing_2::NamedTuple, λ::Vector, params_rb::NamedTuple, h::Float64)::Float64
#   """
#   # Calculate discrete Lagrangian of the system

#   # Arguments
#   - `state_1`: NamedTuple containing states of the system at time t
#   - `state_2`: NamedTuple containing states of the system at time t + h
#   - `forcing_1`: NamedTuple containing forcing terms at time t
#   - `forcing_2`: NamedTuple containing forcing terms at time t + h
#   - `λ`: lagrange multiplier
#   - `params_rb`: NamedTuple containing rigidbody parameters
#   - `h`: time step

#   # Returns
#   - `Ld`: residual of the discrete Lagrangian of the system using midpoint integration
#   """
#   Q1, Q2 = state_1.ᴺQᴮ, state_2.ᴺQᴮ
#   F1, F2 = forcing_1.τᴺ, forcing_2.τᴺ
#   J = params_rb.J
#   # calculate the discrete lagrangian
#   rDEL = ((2/h) * (G(Q2)' * L(Q1) * H * J * H' * L(Q1)' * Q2 + 
#                             G(Q2)' * T * R(Q3)' * H * J * H' * L(Q2)' * Q3) + 
#                    (h/2) * (F1 + F2) + h * Dc(state_2) * λ)

#   return rDEL
# end

# function constraint_revolute_z(state_rb1::NamedTuple, state_rb2::NamedTuple, params_rb1::NamedTuple, params_rb2::NamedTuple)::Vector
#   """
#   # Revolute joint constraint along z-axis

#   # Arguments
#   - `state_rb1`: NamedTuple containing states of the first rigidbody
#   - `state_rb2`: NamedTuple containing states of the second rigidbody
#   - `params_rb1`: NamedTuple containing rigidbody parameters of the first rigidbody

#   # Returns
#   - `C`: constraint residual
#   """
#   # retrieve cg
#   q1, q2 = state_rb1.qᴮ, state_rb2.qᴮ
#   # retrieve joint pos wrt cg
#   cg2joint2_1 = params_rb1.cg2joint[2]
#   cg2joint1_2 = params_rb2.cg2joint[1]
#   # calculate joint position in intertial frame
#   rb1_joint_pos = q1 + quat2rot(state_rb1.ᴺQᴮ) * cg2joint2_1 
#   rb2_joint_pos = q2 + quat2rot(state_rb2.ᴺQᴮ) * cg2joint1_2
#   # calculate constraint residual
#   joint_pos_constraint = rb1_joint_pos - rb2_joint_pos
#   # calculate rotation angle constraint residual
#   joint_rot_constraint = [0 1 0 0; 0 0 1 0] * L(Q1)' * Q2 # extract axis angle and enforce only free rotation in z

#   C = [joint_pos_constraint; joint_rot_constraint]
#   return C
# end